## Load and sort data

In [1]:
import sys
sys.path.insert(0, "../../../")
from dataAnalysis.DataAnalysis import DataAnalysis
import pandas as pd
import torch
 
data = pd.read_csv(r"../../../extdata/sbcdata.csv", header=0)
data_analysis = DataAnalysis(data)

/home/dwalke/git/sbc/graph_learning/patient_centric/positional_encodings/../../../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']
/home/dwalke/git/sbc/graph_learning/patient_centric/positional_encodings/../../../dataAnalysis/data/Filter.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data['Label'] = self.data['Diagnosis']
/home/dwalke/git/sbc/graph_learning/patient_centric/positional_encodings/../../../dataAnalysis/data/Filt

In [2]:
data = pd.concat((data_analysis.get_training_data(), data_analysis.get_testing_data()))
max_Id = data["Id"].unique().max()
gw_data = data_analysis.get_gw_testing_data().copy(deep=True)
gw_data = gw_data.assign(Id=lambda x: x.Id + max_Id)
data = pd.concat((data, gw_data))
data = data.sort_values(["Id", "Time"])
data = data.reset_index(drop=True)
popped_index = data.pop("index")

## Positional encodings

In [5]:
import numpy as np
from dataAnalysis.Constants import FEATURES

def getPositionEncoding(seq_len, d = len(FEATURES), n=10000):
    P = np.zeros((seq_len, d))
    for k in range(seq_len):
        for i in np.arange(int(d/2)):
            denominator = np.power(n, 2*i/d)
            P[k, 2*i] = np.sin(k/denominator)
            P[k, 2*i+1] = np.cos(k/denominator)
    return P

pos_encodings = None

for Id, group in data.groupby("Id"):
    encoding = getPositionEncoding(group.shape[0])
    pos_encodings = encoding if pos_encodings is None else np.concatenate((pos_encodings, encoding), axis=0)
   
pd.DataFrame(pos_encodings).to_csv("positional_encodings.csv", index=False)

In [4]:
pos_encodings

array([[ 0.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.7568025 , -0.65364362,  0.28391461, ...,  0.02071642,
         0.99978539,  0.        ],
       [ 0.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [ 0.84147098,  0.54030231,  0.07190646, ...,  0.00517945,
         0.99998659,  0.        ]])

In [ ]:
pos